# Desenvolvendo um corretor ortográfico

Neste projeto utilizei algumas técnicas de **NLP** e lógica de programação em python para desenvolver um modelo capaz de corrigir aluns erros de digitação, como caracteres faltantes ou a mais, caracteres digitados errados ou invertidos.

Ao final do projeto foram desenvolvidos 2 corretores, um com taxa de acerto de 55% e o outro com acerto de 76%, algo que ainda pode ser melhorado inserindo mais exemplos de palavras para aumentar o vocabulário do corretor.

- Importando a biblioteca utilizada

In [1]:
import nltk

- Abrindo e exibindo os primeiros registros do arquivo usado como vocabulário para o corretor de palavras.

In [2]:
with open('artigos.txt','r', encoding="utf8") as f:
  artigos = f.read()

print(artigos[:100])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar u


**Tokenizando o texto**

- A função abaixo serve para retornar apenas as  palavras de um texto, onde foi feito a tokenização e padronização para que retorne todas as palavras separadas e com escrita em letras minúsculas.

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Biena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#  função que recebe um texto retorna apenas uma lista com palavras
def tokenizacao(texto):
  lista_de_palavras=[]
  lista_tokens = nltk.tokenize.word_tokenize(texto)
  lista_de_tokens = [palavra.lower() for palavra in lista_tokens]
  for token in lista_de_tokens:
    if token.isalpha():
      lista_de_palavras.append(token)

  return lista_de_palavras

- Exempplo da função de tokenização e normalização funcionando

In [5]:
teste='OLÁ, SEJA BEM VINDO ao projeto corretor ortográfico utilizando Técnicas DE  NlP'

tokenizacao(teste)

['olá',
 'seja',
 'bem',
 'vindo',
 'ao',
 'projeto',
 'corretor',
 'ortográfico',
 'utilizando',
 'técnicas',
 'de',
 'nlp']

- Verificando q quantidade de palavras no arquivo

In [6]:
# quantidade de palavras no arquivo artigos.txt
 
print(len(tokenizacao(artigos)))

403106


In [7]:
#  removendo palavras repetidas
print(len(set(tokenizacao(artigos))))

18465


# Desenvolvendo a função corretor ortográfico

##  A função corretor  será desenvolvida por meio de outras funções, sendo elas:

**Inserir letras**: onde simula casos em que esquecemos se alguma letra na digitação das palavras

**Remover letras**: onde simula casos em que digitamos letras a mais. 

**Trocar letras** onde simula casos de escrita de alguma letra errada errada, e a função irá subistituir letra por letra da palavra por outras letras e formando novas palavras.

**Inverte letras** onde simula casos de escrita de alguma letra na posição errada, e a função irá inverter a posição das letras na palavra formando novas palavras.

**Gerador de palavras** esta função abrange as 4 funções acima e gera um conjunto de palavras possíveis a partir da palavra digitada e que deve ser corrigida.


A performance do corretor será avaliada calculando a probabilidade das palavras geradas, dividindo pela quatidade total de palavras que tem no arquivo.

In [8]:
#  criando função para fatiar e criar novas palavras
def inserir_letras(fatias):
  novas_palavras=[]
  letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for E,D in fatias:
    for letra in letras:
      novas_palavras.append(E+ letra + D)

  return novas_palavras

def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáàãâéèêíìóòõôúùûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D)>1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

def gerador_de_palavras(palavra):
  fatias=[]
  for i in range(len(palavra)+1):
    fatias.append((palavra[:i], palavra[i:]))

  palavras_geradas=inserir_letras(fatias)
  palavras_geradas+=deletando_caracteres(fatias)
  palavras_geradas+=troca_letra(fatias)
  palavras_geradas+=inverte_letra(fatias)
  
  return palavras_geradas

Nesta etapa foram criadas as funções:

**Probabilidade** que utiliza a frequência de cada palavra  dividido pela quantidade total de palavrass do arquivo.

**Corretor** que utilizada a função probalbilidade para em meio as palavras eradas selecionar a que tem maior probabilidade e inidicar como sendo a palavra correta.

**Cria dados teste** que cria uma lista de dados de teste para avaliar o corretor.

**Avaliador** que avalia a performance do correto de palavras, dando a taxa de acerto e a % de palavras que não estão no vocabulário do corretor.



In [9]:
# lista com todas as palavras separadas e normalizadas do arquivo "artigos.txt"
lista_normalizada =  tokenizacao(artigos)

# calculando a frequencia de cada palavra usando o método FreqDist e xibindo as 10 palavras mais comuns
frequencia = nltk.FreqDist(lista_normalizada)

frequencia.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6368),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [10]:
total_palavras=len(lista_normalizada)

In [11]:
# criando a função probalibilidade para saber a probabilidade de cada palavra
def probabilidade(palavra_gerada):
    probabilidade=frequencia[palavra_gerada]/total_palavras
    return probabilidade

# criando a função corretor de palavras
def corretor(palavra):
    palavras_geradas = gerador_de_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [12]:
# criando dados de teste para avaliar o corretor
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, "r",encoding='utf-8')
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

In [13]:
lista_teste = cria_dados_teste("palavras.txt")
lista_teste[:10]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm')]

In [14]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acerto = 0
    desconhecida = 0
    for correta, errada in testes:
        desconhecida += (correta not in vocabulario)
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acerto += 1
    
    taxa_acerto = round(acerto*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'''Taxa de acerto é {taxa_acerto} % de {numero_palavras} palavras\n'''
          f'''A porcentagem de palavras desconhecidas é de {taxa_desconhecida}%''')


vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

Taxa de acerto é 76.34 % de 186 palavras
A porcentagem de palavras desconhecidas é de 6.99%


# Implementando o corretor de palavras com  2 correções


In [15]:
#  criando um novo gerador de palavras
def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_de_palavras(palavra)
    return novas_palavras

palavra = "lóiigica"
palavras_g = gerador_turbinado(gerador_de_palavras(palavra))
"lógica" in palavras_g

True

In [16]:
# criando um novo corretor usando o corretor já desenvolvido
def novo_corretor(palavra):
    palavras_geradas = gerador_de_palavras(palavra)
    palavras_turbinado = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbinado)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
            candidatos.append(palavra)
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

In [17]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acerto = 0
    desconhecida = 0
    for correta, errada in testes:
        desconhecida += (correta not in vocabulario)
        palavra_corrigida = novo_corretor(errada)
        if palavra_corrigida == correta:
            acerto += 1
    
    taxa_acerto = round(acerto*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'''Taxa de acerto é {taxa_acerto} % de {numero_palavras} palavras\n'''
          f'''A porcentagem de palavras desconhecidas é de {taxa_desconhecida}%''')


vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

Taxa de acerto é 55.38 % de 186 palavras
A porcentagem de palavras desconhecidas é de 6.99%


In [18]:
# palavra correta="lógica"
palavra='lgica'
print(f'Coreção da palavra {palavra} do 1º corretor: "{corretor(palavra)}" ')
print(f'Correção da palavra {palavra} do 2º corretor: "{novo_corretor(palavra)}""')

Coreção da palavra lgica do 1º corretor: "lógica" 
Correção da palavra lgica do 2º corretor: "fica""


In [19]:
# palavra correta="lógica"
palavra='lóiigica'
print(f'Coreção da palavra {palavra} do 1º corretor: "{corretor(palavra)}" ')
print(f'Correção da palavra {palavra} do 2º corretor: "{novo_corretor(palavra)}""')

Coreção da palavra lóiigica do 1º corretor: "alóiigica" 
Correção da palavra lóiigica do 2º corretor: "lógica""
